# M03 Homework

```yaml
Course:  DS 5001
Module:  M03 Language Models
Author:  JiHo Lee (qcx6hb)
Date:    4 February 2024
```

### For setup

In [5]:
import pandas as pd
import numpy as np
import configparser

config = configparser.ConfigParser()
config.read("../../../env.ini")
data_dir = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

data_dir  = data_dir.replace('/', '\\')
output_dir  = output_dir.replace('/', '\\')

ngrams = 2
widx = [f"w{i}" for i in range(ngrams)]

# data_dir = output_dir
OHCO = ['chap_id','para_num','sent_num','token_num']

text_file = f"{data_dir}/gutenberg/pg42324.txt"
csv_token  = f"{output_dir}/austen-combo-TOKENS.csv" # The file we will create
csv_vocab = f"{output_dir}/austen-combo-VOCAB.csv" 

text_file = text_file.replace('/', '\\')
csv_token = csv_token.replace('/', '\\')
csv_vocab = csv_vocab.replace('/', '\\')

from textimporter import TextImporter
from textparser import TextParser

src_file = text_file
# ohco_pats = [('chap', r'^(?:INTRODUCTION|PREFACE|LETTER|CHAPTER)\.?\b', 'm')]
ohco_pats = [('chap', r'^(?:INTRODUCTION|PREFACE|LETTER|CHAPTER)\b', 'm')]
clip_pats = [r'START', r'END']

test= TextImporter(src_file=src_file, ohco_pats=ohco_pats, clip_pats=clip_pats)
test.import_source().parse_tokens()

TOKEN = test.TOKENS
test.extract_vocab()
VOCAB = test.VOCAB

# VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['modified_term_str'] = VOCAB.index
VOCAB.loc[(VOCAB.n == 1) & (VOCAB.n_chars < 3), 'modified_term_str'] = "<UNK>"

TOKEN['modified_term_str'] = TOKEN.term_str.map(VOCAB.modified_term_str)
# TOKEN[TOKEN.modified_term_str == '<UNK>'].sample(5)



Importing  C:\DS5001\DS5001_2024_01_R\..\data\gutenberg\pg42324.txt
Clipping text
Parsing OHCO level 0 chap_id by milestone ^(?:INTRODUCTION|PREFACE|LETTER|CHAPTER)\b
Parsing OHCO level 1 para_num by delimitter \n\n
Parsing OHCO level 2 sent_num by delimitter [.?!;:]+
Parsing OHCO level 3 token_num by delimitter [\s',-]+


In [6]:
def token_to_padded(token, grouper=['sent_num'], term_str='term_str'):
    ohco = token.index.names # We preserve these since they get lost in the shuffle
#     print(ohco)
    padded = token.groupby(grouper)\
        .apply(lambda x: '<s> ' + ' '.join(x[term_str]) + ' </s>')\
        .apply(lambda x: pd.Series(x.split()))\
        .stack().to_frame('term_str')
#     print(padded)
#     print(ohco)
    padded.index.names = ohco 
    return padded
def padded_to_ngrams(padded, grouper=['sent_num'], n=2):
    
    ohco = padded.index.names
    ngrams = padded.groupby(grouper)\
        .apply(lambda x: pd.concat([x.shift(0-i) for i in range(n)], axis=1))\
        .reset_index(drop=True)
    ngrams.index = padded.index
    ngrams.columns = widx

    # ngrams = pd.concat([padded.shift(0-i) for i in range(n)], axis=1)
    # ngrams.index.name = 'ngram_num'
    # ngrams.columns = widx
    # ngrams = ngrams.fillna('<EOF>')
    
    return ngrams
PADDED = token_to_padded(TOKEN, grouper=OHCO[:3], term_str='modified_term_str')
NGRAMS = padded_to_ngrams(PADDED, OHCO[:3], ngrams)

### Question 1.
1. List six words that precede the word "monster," excluding stop words (and sentence boundary markers). Stop words include 'a', 'an', 'the', 'this', 'that', etc. Hint: use the df.query() method.

Answers: miserable, abhorred, detestable, hideous, hellish, gigantic

In [7]:
stop = ['this', 'the', 'a', '<s>']
tmp = NGRAMS.query("w1=='monster'")
tmp2 = tmp[~tmp['w0'].isin(['this', 'the', 'a', '<s>'])]
tmp2

w0       w1
chap_id para_num sent_num token_num                     
13      3        17       25          miserable  monster
18      8        0        1            abhorred  monster
23      25       4        23         detestable  monster
24      28       0        1             hideous  monster
32      4        9        5             hellish  monster
        17       6        2            gigantic  monster

### Question 2.
2. List the following sentences in ascending order of bigram perplexity according to the language model generated from the text:

Answers:

I have never seen the aurora borealis.	

The monster is on the ice.	

He never knew the love of a family.

Flowers are happy things.	


In [8]:
def ngrams_to_models(ngrams):
    global widx
    n = len(ngrams.columns)
    model = [None for i in range(n)]
    for i in range(n):
        if i == 0:
            model[i] = ngrams.value_counts('w0').to_frame('n')
            model[i]['p'] = model[i].n / model[i].n.sum()
            model[i]['i'] = np.log2(1/model[i].p)
        else:
            model[i] = ngrams.value_counts(widx[:i+1]).to_frame('n')    
            model[i]['cp'] = model[i].n / model[i-1].n
            model[i]['i'] = np.log2(1/model[i].cp)
        model[i] = model[i].sort_index()
    return model
M = ngrams_to_models(NGRAMS)

S= "The monster is on the ice.\nFlowers are happy things.\nI have never seen the aurora borealis.\nHe never knew the love of a family."
S = S.split('\n')
S = pd.DataFrame(S, columns=['sent_str'])
S.index.name='sent_num'

# Convert dataframe of sentences to TOKEN with normalized terms
K = S.sent_str.apply(lambda x: pd.Series(x.split())).stack().to_frame('token_str')
K['term_str'] = K.token_str.str.replace(r"[\W_]+", "", regex=True).str.lower()
K.index.names = ['sent_num', 'token_num']

TEST_SENTS, TEST_TOKENS = S, K
TEST_TOKENS.loc[~TEST_TOKENS.term_str.isin(M[0].index), 'term_str'] = "<UNK>"
# TEST_TOKENS[TEST_TOKENS.term_str == '<UNK>'].value_counts('token_str')
TEST_PADDED = token_to_padded(TEST_TOKENS)
TEST_NGRAMS = padded_to_ngrams(TEST_PADDED, 'sent_num', 2)

def test_model(model, ngrams, sents):
    
    global widx
    
    assert len(model) == len(ngrams.columns)
    
    n = len(model)
    ohco = ngrams.index.names
    
    R = []
    for i in range(n):
        T = ngrams.merge(M[i], on=widx[:i+1], how='left')
        T.index = ngrams.index
        T = T.reset_index().set_index(ohco + widx).i #.to_frame(f"i{i}")
        
        # This how we handle unseen combos
        T[T.isna()] = T.max()
        R.append(T.to_frame(f"i{i}"))
                
    return pd.concat(R, axis=1)
R = test_model(M,TEST_NGRAMS, TEST_SENTS)
def compute_perplexity(results, test_sents, n=2):
    for i in range(n):
        test_sents[f"pp{i}"] = np.exp2(results.groupby('sent_num')[f"i{i}"].mean())
    return test_sents

PP = compute_perplexity(R, TEST_SENTS)

In [11]:
PP.sort_values(by=['pp1'])

,sent_str,pp0,pp1
sent_num,,,
2,I have never seen the aurora borealis.,299.843636,36.637991
0,The monster is on the ice.,116.931101,72.914436
3,He never knew the love of a family.,171.859389,117.836895
1,Flowers are happy things.,552.541874,358.229326


### Question 3.
3. Using the bigram model represented as a matrix, explore the relationship between bigram pairs using the following lists. Hint: use the .unstack() method on the feature n and then use .loc[] to select the first list from the index, and the second list from the columns.
    ['he','she'] to select the indices.
    ['said','heard'] to select the columns.

In [32]:
df3 = NGRAMS.pp1.unstack(fill_value=0)
# df3.style.background_gradient(cmap='YlGnBu', axis=None)

AttributeError: 'DataFrame' object has no attribute 'pp1'

### Question 4.
4. Generate 20 sentences using the generate_text() function. Display the results.

In [21]:
ngrams = 3
widx = [f"w{i}" for i in range(ngrams)]

NGRAMS = padded_to_ngrams(PADDED, OHCO[:3], ngrams)
M = ngrams_to_models(NGRAMS)

# TEST_PADDED = token_to_padded(TEST_TOKENS)
# TEST_NGRAMS = padded_to_ngrams(TEST_PADDED, 'sent_num', 3)
# M = ngrams_to_models(TEST_NGRAMS)

def generate_text(M, n=250):
    
    if len(M) < 3:
        raise ValueError("Must have trigram model generated.")
    
    # Start list of words
    first_word = M[1].loc['<s>'].sample(weights='cp').index[0]
    
    words = ['<s>', first_word]
    
    for i in range(n):
        
        bg = tuple(words[-2:])

        # Try trigram model
        try:
            next_word = M[2].loc[bg].sample(weights='cp').index[0]

        # If not found in model, back off ...
        except KeyError as e1:
            try:
                # Get the last word in the bigram
                ug = bg[1]
                next_word = M[1].loc[ug].sample(weights='cp').index[0]
            
            except KeyError as e2:
                next_word = M[0].sample(weights='p').index[0]
                
        words.append(next_word)
    
    
    text = ' '.join(words[2:])
    print('\n\n'.join([str(i+1) + ' ' + line.replace('<s>','')\
        .strip().upper() for i, line in enumerate(text.split('</s>'))]))
# generate_text(M)
generate_text(M)

1 HINDOOS GIVE THE WORLD

2 MIEN DURING HIS LECTURE HAD REMOVED MY PREJUDICES AGAINST MODERN CHEMISTS

3 LISTEN TO ME HANGING AMONG THE TREES

4 I HAD LEARNED FROM THE THROATS OF THE STARS OFTEN DISAPPEARED IN THE ALPS OF SAVOY

5 THE CAUSE OF THIS REMARK MY DEAR VICTOR IS NECESSARY TO CALM HER MIND

6 AND THE SURROUNDING MOUNTAINS

7 

8 TUTORED AND REFINED BY BOOKS AND RETIREMENT FROM THE INTERCOURSE OF THE MOUNTAIN OR TRANSVERSELY UPON OTHER TREES

9 ALSO TO FORGET MYSELF AND I THOUGHT THAT MY CONVERSATION HAD INTERESTED THE FATHER OF A PROMONTORY FLOURISHING VINEYARDS WITH GREEN SLOPING BANKS AND A SLAVE BY THE SHOCK BUT ENTIRELY REDUCED TO A CONDUCT SO LITTLE WORTHY OF MY AUNT IN HER ALSO

10 THE PICTURE OF MY NEW PRECEPTORS I ENTERED

11 CONSENTED TO RETURN TO THE PLEASURE I PERCEIVED AS THE SHAPE CAME NEARER SIGHT TREMENDOUS AND OVERWHELMING SOUND

12 CONSISTED ENTIRELY OF THE HUMAN FRAME AND INDEED NONE OF OUR GOOD UNCLE THOMAS S LIBRARY

13 WAR AGAINST THE PANES AND MY DREAMS 

### Question 5.
5. Compute the redundancy 
 for each of the n-gram models using the MLE of the joint probability of each ngram type. In other words, for each model, just use the .mle feature as p 
 in computing H.
 
 - Does R increase, decrease, or remain the same as the choice of n-gram increases in length?
 - Hint: Remember that R, where H is the actual entropy of the model and H_max is its maximum entropy
 
 - If mle is not a feature in your model, just use p for the unigram model and compute p for the other two models by dividing n by the sum of n
 - N is computed as the number of all possible combinations for each ngram. So, for the bigram model N is the number of unigrams (i.e. the vocabulary size plus the sentence boundary signs) squared, and for the trigram model the value is cubed, 


In [28]:
NGRAMS

w0          w1          w2
chap_id para_num sent_num token_num                                    
1       0        0        0                 <s>         the  publishers
                          1                 the  publishers          of
                          2          publishers          of         the
                          3                  of         the    standard
                          4                 the    standard      novels
...                                         ...         ...         ...
32      82       1        11                 in    darkness         and
                          12           darkness         and    distance
                          13                and    distance        </s>
                          14           distance        </s>        None
                          15               </s>        None        None

[88801 rows x 3 columns]

In [23]:
VOCAB
n_terms = VOCAB.n.count() # same as vocab.shape[0]
H = VOCAB.h.sum()
Hmax = np.log2(n_terms)
R = 1 - (H/Hmax)

In [25]:
R

0.2733562891486093